In [17]:
# Loading Required Packages 
import cobra
import pandas as pd
import numpy as np

# Loading the Model 
model = cobra.io.read_sbml_model("ArabidopsisCoreModel1.xml")

# Changing the directionality of Fum_c to Mal_c to go the other way (day-time model) 
r = model.reactions.get_by_id('FumHA_c')
r.add_metabolites({model.metabolites.get_by_id('Fum_c'): +2})
r.add_metabolites({model.metabolites.get_by_id('H2O_c'): +2})
r.add_metabolites({model.metabolites.get_by_id('Mal_c'): -2})
print(r.reaction)
print(r.bounds)
print("Directionality changed. Dyson et al. 2016 \n")

# Creating a Malate "Storage" Reactions
r = cobra.Reaction('Mal_Store')
r.name = 'Mal_Store'
r.add_metabolites({model.metabolites.get_by_id('Mal_c'): -1})
model.add_reaction(r)
print(r.reaction)

# Creating a Fumarate "Storage" Reactions
r = cobra.Reaction('Fum_Store')
r.name = 'Fum_Store'
r.add_metabolites({model.metabolites.get_by_id('Fum_c'): -1})
model.add_reaction(r)
print(r.reaction)

# Creating a Starch "Storage" Reactions
r = cobra.Reaction('Starch_Store')
r.name = 'Starch_Store'
r.add_metabolites({model.metabolites.get_by_id('starch1_h'): -1})
model.add_reaction(r)
print(r.reaction)

cobra/core/reaction.py:394 UserWarning: malformed gene_reaction_rule '2*(ATCG00020 and ATCG00680 and ATCG00280 and ATCG00270 and ATCG00580 and ATCG00570 and ATCG00710 and ATCG00080 and ATCG00550 and ATCG00070 and ATCG00560 and ATCG00220 and ATCG00700 and (AT5G66570 or AT3G50820) and AT1G06680 and (AT4G21280 or AT4G05180) and AT1G79040 and AT1G44575 and ATCG00690 and AT3G21055 and AT2G30570 and AT2G06520 and AT1G67740 and ATCG00300)' for <Reaction PSII_h at 0x7fb944e22110>
cobra/core/reaction.py:394 UserWarning: malformed gene_reaction_rule '2*(ATCG00540 and ATCG00720 and AT4G03280 and ATCG00730 and ATCG00600 and ATCG00590 and AT2G26500 and ATCG00210)' for <Reaction Cytb6f_h at 0x7fb944ed7590>
cobra/core/reaction.py:394 UserWarning: malformed gene_reaction_rule '8*(ATCG00490) and 8*(AT5G38430 or AT5G38420 or AT1G67090 or AT5G38410)' for <Reaction RBC_h at 0x7fb944e22150>
cobra/core/reaction.py:394 UserWarning: malformed gene_reaction_rule '4*(AT3G26650 or AT1G12900) or (2*(AT3G26650 or 

Mal_c --> Fum_c + H2O_c
(0.0, 1000.0)
Directionality changed. Dyson et al. 2016 

Mal_c --> 
Fum_c --> 
starch1_h --> 


In [18]:
# Get list of all possible proteins in the model
gene_ids = []
for g in model.genes:
    gene_ids.append(g.id)

In [19]:
# Reading in the protein data
df = pd.read_excel(io="ProteinConc.xlsx")

In [20]:
# Get list of all measured proteins 
all_atg = []
x=df["Accession"]
for atg in x.values:
    all_atg.append(atg)

In [21]:
# Comparing the measured protein concentrations to those which are in the model 

model_atg = [] # Total measured proteins in model
model_atg_final = [] # Proteins with direct reaction constraint (i.e. single responsible protein)
model_rxns_atg = [] # Model reactions with direct constraints (i.e. single responsible protein)
model_rxns_atgs = [] # Model reactions with multiple possible constraints 
model_atgs_final = [] # Proteins with multiple possible constraints 

for atg in x.values: # Check if protein is in the model 
    if atg in gene_ids:
        model_atg.append(atg)
        # Only parameterize that reaction if that enzyme is the only one responsible for that reaction
        for r in model.genes.get_by_id(atg).reactions:
            if len(r.genes) == 1:
                model_rxns_atg.append(r)
                model_atg_final.append(atg)
            else:
                prot_names = []
                for g in r.genes:
                    prot_names.append(g.id)
                if set(prot_names).issubset(set(all_atg)):
                    model_rxns_atgs.append(r)
                    model_atgs_final = model_atgs_final + prot_names

# Feasible constraint summary 
# General 
print("Total measured proteins: {}".format(len(all_atg)))  
print("Total measured proteins in model: {}".format(len(model_atg))) 
print("Total model reactions: {}".format(len(model.reactions)))
print(" ")
# Reactions with simple constraints 
print("Total model reactions with single constraints: {}".format(len(set(model_rxns_atg))))
print("Model proteins for single constraints: {}".format(len(set(model_atg_final))))
print(" ")
# Reactions with multiple possible constraints 
print("Total model reactions with multiple constraints: {}".format(len(set(model_rxns_atgs))))
print("Total model proteins for possible constraints: {}".format(len(set(model_atgs_final))))

Total measured proteins: 2427
Total measured proteins in model: 245
Total model reactions: 552
 
Total model reactions with single constraints: 62
Model proteins for single constraints: 49
 
Total model reactions with multiple constraints: 39
Total model proteins for possible constraints: 53


In [22]:
# Function for constraining the model reactions for a specific genotype and temperature

def constr(reacs,Dat1,Dat2,Dat3,Dat4):
    for r in reacs:
        upper_vals = []
        lower_vals = []
        for g in r.genes:
            # Get index of protein in data file 
            ind = all_atg.index(g.id)
            # Scaling the values for setting fluxes 
            vals = [i/4000. for i in Dat1[ind],Dat2[ind],Dat3[ind],Dat4[ind]]
            val = np.mean(vals)
            sterr = np.std(vals)/np.sqrt(len(vals))
            upper = val + sterr
            lower = val - sterr
            upper_vals.append(upper)
            lower_vals.append(lower)
        # Constraint flux of forward reaction
        r.upper_bound = round(sum(upper_vals),3)
        # Constraint flux of lower reaction 
        if r.lower_bound < 0.0:
            r.lower_bound = round(-sum(upper_vals),3)
        # No lower bound b/c don't know localization or enzymes are used to in other reactions 

In [ ]:
# RUN THIS NEXT SECTION AS EITHER OR! RESERT KERNEL EACH TIME. 

In [ ]:
# Parameterizing the model for Col-0 Control Conditions

# Set up reactions which will be constrained
constr_reacs = set(model_rxns_atg + model_rxns_atgs)

# Constraining the model
constr(constr_reacs,df["WT Cntl 1"],df["WT Cntl 2"],df["WT Cntl 3"],df["WT Cntl 4"])
    
# Saving the model
cobra.io.write_sbml_model(model,"Arnold2014_WT_Cntl.xml",use_fbc_package=False)

In [16]:
# Parameterizing the model for Fum2 Control Conditions

# Set up reactions which will be constrained
constr_reacs = set(model_rxns_atg + model_rxns_atgs)

# Constraining the model
r = model.reactions.get_by_id('FumHA_c')
r.upper_bound = 0.0 # Mutant knockout 
r.lower_bound = 0.0 
constr(constr_reacs,df["fum2 Cntl 1"],df["fum2 Cntl 2"],df["fum2 Cntl 3"],df["fum2 Cntl 4"]) #Proteomics 
    
# Saving the model
cobra.io.write_sbml_model(model,"Arnold2014_M_Cntl.xml",use_fbc_package=False)

In [ ]:
# Parameterizing the model for Col-0 Control Conditions

# Set up reactions which will be constrained
constr_reacs = set(model_rxns_atg + model_rxns_atgs)

# Constraining the model
constr(constr_reacs,df["WT cold 1"],df["WT cold 2"],df["WT cold 3"],df["WT cold 4"])
    
# Saving the model
cobra.io.write_sbml_model(model,"Arnold2014_WT_Cold.xml",use_fbc_package=False)

In [23]:
# Parameterizing the model for Fum2 Cold Conditions

# Set up reactions which will be constrained
constr_reacs = set(model_rxns_atg + model_rxns_atgs)

# Constraining the model
r = model.reactions.get_by_id('FumHA_c')
r.upper_bound = 0.0 # Mutant knockout 
r.lower_bound = 0.0 
constr(constr_reacs,df["fum2 Cold 1"],df["fum2 Cold 2"],df["fum2 Cold 3"],df["fum2 Cold 4"]) #Proteomics 
    
# Saving the model
cobra.io.write_sbml_model(model,"Arnold2014_M_Cold.xml",use_fbc_package=False)

In [ ]:
# CHRR flux sampling (MATLAB SCRIPT)
# Plotting the overlapping sampling distribution (NEW PYTHON SCRIPT)